# Upload synthetics to dandi

author: laquitainesteeve@gmail.com

Execution time: 10 min

Special hardware: on CPU, does not require GPU.

# Setup 

Activate demo virtual environment (envs/demo.yml)

```bash
python -m ipykernel install --user --name demo --display-name "demo"
```

In [1]:
%%time 

# import python packages
import os
import yaml
import numpy as np
from time import time
from dandi.dandiapi import DandiAPIClient
import spikeinterface.extractors as se
import spikeinterface.sorters as ss
import spikeinterface
import spikeinterface as si
from pynwb.file import NWBFile, Subject
from pynwb import NWBHDF5IO
import uuid
from datetime import datetime
from dateutil.tz import tzlocal
from neuroconv.tools.spikeinterface import add_recording_to_nwbfile, add_sorting_to_nwbfile
print("spikeinterface", spikeinterface.__version__)

# setup project
proj_path = "/home/steeve/steeve/epfl/code/spikebias/"
os.chdir(proj_path)

from src.nodes.dandi import write_nwb

# setup pipeline parameters
DANDISET_ID = '001250'

# setup nwb files save paths
FILE_PATH = os.path.join(proj_path, "temp/pros_of_details/")

/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


spikeinterface 0.101.2
CPU times: user 3.99 s, sys: 1.92 s, total: 5.9 s
Wall time: 725 ms


## Custom functions

In [ ]:
def upload_recording_and_ground_truth_nwb_to_dandi(dandiset_id: str, subject_path: str, file_path: str):
    """Upload the recording and ground truth nwb datasets 
    to dandi archive
    """

    # download a minimal local dandiset with only the .yaml config
    os.system(f"dandi download --download dandiset.yaml --output-dir {file_path} DANDI:{dandiset_id}")

    # go to path of the dandiset
    os.chdir(file_path)

    # upload
    os.system(
        f"""
        export DANDI_API_KEY='210e68743286d64e84743bd8980d5771ef82bf4d';
        cd {os.path.join(file_path, dandiset_id)};
        dandi organize {subject_path} -f dry;
        dandi organize {subject_path};    
        dandi upload
        """
    )

    # delete local dandiset
    os.system(f"rm -rf {os.path.join(file_path, dandiset_id)}")

## Upload buccino replicate

- This replicates Buccino et al., simulation with MEAREC as described in the Spikeinterface paper
- 2.8 GB on dandi

In [ ]:
%%time 

# setup parameters
SUBJECT_ID = "mearec-synthetics"
SESSION_ID = 'buccino_replicate'
SESSION_DESCRIPTION = 'Replication of Buccino synthetic model'        
RECORDING_PATH = 'dataset/00_raw/recording_buccino_rep/'
GROUND_TRUTH_PATH = 'dataset/00_raw/ground_truth_buccino_rep/'
SUBJECT_PATH = os.path.join(FILE_PATH, f"{DANDISET_ID}/sub-{SUBJECT_ID}/")
NWB_FILE_PATH = os.path.join(SUBJECT_PATH, f"sub-{SUBJECT_ID}_ses-{SESSION_ID}.nwb")
PARAMS = {
    "subject_id": SUBJECT_ID,
    "session_description": SESSION_DESCRIPTION,
    "identifier": str(uuid.uuid4()),
    "session_start_time": datetime.now(tzlocal()),
    "experimenter": "Laquitaine Steeve",
    "lab": "Blue Brain Project",
    "institution": "EPFL",
    "experiment_description": SESSION_DESCRIPTION,
    "session_id": SESSION_ID,
    "related_publications": "doi:"
    }

# load extractors
RecordingS1 = si.load_extractor(RECORDING_PATH)
SortingGtS1 = si.load_extractor(GROUND_TRUTH_PATH)
    
# write as nwb
write_nwb.run(RECORDING_PATH, GROUND_TRUTH_PATH, NWB_FILE_PATH, PARAMS)

# upload
upload_recording_and_ground_truth_nwb_to_dandi(DANDISET_ID, SUBJECT_PATH, FILE_PATH)

2025-08-13 15:04:43,195 - root - write_nwb.py - run - INFO - Ground truth metadata:
2025-08-13 15:04:43,196 - root - write_nwb.py - run - INFO - ['z', 'y', 'bursting', 'mtype', 'exp_decay', 'x', 'max_burst_duration', 'max_spikes_per_burst', 'cell_type', 'snr']


/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.5 is recommended
  warnings.warn(
/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/neuroconv/tools/spikeinterface/spikeinterface.py:1054: UserWarning: The 'add_recording' function is deprecated and will be removed after February 2025. 
Please use 'add_recording_to_nwbfile' instead.
  warnings.warn(
/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/neuroconv/tools/spikeinterface/spikeinterface.py:1676: UserWarning: The 'add_sorting' function is deprecated and will be removed after February 2025. 
Please use 'add_sorting_to_nwbfile' instead.
  warnings.warn(


PATH                 SIZE DONE    DONE% CHECKSUM STATUS MESSAGE   
001250/dandiset.yaml                             done   updated   
Summary:                  0 Bytes                1 done 1 updated 
                          <0.00%                                  


2025-08-13 15:04:47,567 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.08.13-13.04.45Z-3405129.log
2025-08-13 15:04:48,364 [    INFO] Loading metadata from 1 files
2025-08-13 15:04:48,482 [    INFO] Organized 0 out of 1 paths. Visit /home/steeve/steeve/epfl/code/spikebias/temp/pros_of_details/001250/
2025-08-13 15:04:48,482 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.08.13-13.04.47Z-3405161.log


DRY: /home/steeve/steeve/epfl/code/spikebias/temp/pros_of_details/001250/sub-mearec-synthetics/sub-mearec-synthetics_ses-buccino_replicate.nwb -> sub-mearec-synthetics/sub-mearec-synthetics_ecephys.nwb


2025-08-13 15:04:49,329 [    INFO] Loading metadata from 1 files
2025-08-13 15:04:49,330 [    INFO] Symlink support autodetected; setting files_mode='symlink'
2025-08-13 15:04:49,333 [    INFO] Organized 1 paths. Visit /home/steeve/steeve/epfl/code/spikebias/temp/pros_of_details/001250/
2025-08-13 15:04:49,333 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.08.13-13.04.48Z-3405234.log
2025-08-13 15:04:50,898 [    INFO] Found 3 files to consider
